# Tips & Trics for anypytools

## Stopping and restarting simulations

AnyPyProcess caches the simulation results. This allows us to stop the simulations, and then later restart them again.

In [12]:
from anypytools.abcutils import AnyPyProcess 
app = AnyPyProcess(num_processes=1, return_task_info = True )
macro = [['load "Knee.any"', 
          'operation Main.MyStudy.InverseDynamics',
          'run']]*20

In [13]:
app.start_macro(macro);

 [*****************85%************      ]  17 of 20 complete

Here we stopped the simulation using the Notebook interrupt button. Calling the `start_macro()` function again continues the processing and re-run any task that wasn't completed in the first run and any task that exited with errors. 

In [14]:
app.start_macro(macro);

 [****************100%******************]  20 of 20 complete

Note: That changing the input arguments `start_macro` or re-instanciating the the `app` object will erase the cache and re-run all processes. 

## Including meta-information in the output

The typical behaviour of `start_macro()` is to only return variables 'dump'ed by the macro commands. 

In [15]:
from anypytools.abcutils import AnyPyProcess 

app = AnyPyProcess()
macro = [['load "Knee.any" -def N_STEP="10"', 
          'operation Main.MyStudy.InverseDynamics',
          'classoperation Main.MyStudy.Output.MaxMuscleActivity "Dump"',
          'run']]
app.start_macro(macro)

 [****************100%******************]  1 of 1 complete

[{'Main.MyStudy.Output.MaxMuscleActivity': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}]

To get more informatin about the tasks, we can set `return_task_info` flag. 

In [16]:
from anypytools.abcutils import AnyPyProcess 

app = AnyPyProcess(return_task_info = True)
macro = [['load "Knee.any" -def N_STEP="10"', 
          'operation Main.MyStudy.InverseDynamics',
          'classoperation Main.MyStudy.Output.MaxMuscleActivity "Dump"',
          'run']]
app.start_macro(macro) 

 [****************100%******************]  1 of 1 complete

[{'Main.MyStudy.Output.MaxMuscleActivity': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  u'task_id': 0,
  u'task_logfile': u'',
  u'task_macro': ['load "Knee.any" -def N_STEP="10"',
                  'operation Main.MyStudy.InverseDynamics',
                  'classoperation Main.MyStudy.Output.MaxMuscleActivity "Dump"',
                  'run'],
  u'task_macro_hash': 2122630419,
  u'task_name': 'AnyPyTools/Tutorial_0',
  u'task_processtime': 0.5044194832298956,
  u'task_work_dir': 'E:\\Users\\MEL\\GitHub\\AnyPyTools\\Tutorial'}]

That returns extra meta information about the processing and the macros... 

## Saving output to re-process at a later time

Saving data with 'extra' task information, gives some extra options. The output from `start_macro` can be used as input to restart the same processing even if the AnyPyProcess have no cached results. 

In [17]:
from anypytools.abcutils import AnyPyProcess 

app = AnyPyProcess(return_task_info = True)
macro = [['load "Knee.any" -def N_STEP="10"', 
          'operation Main.MyStudy.InverseDynamics',
          'classoperation Main.MyStudy.Output.MaxMuscleActivity "Dump"',
          'run']]
output = app.start_macro(macro) 

 [****************100%******************]  1 of 1 complete

In [18]:
app = AnyPyProcess()
app.start_macro(output)

 [****************100%******************]  1 of 1 complete

[{'Main.MyStudy.Output.MaxMuscleActivity': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  u'task_id': 0,
  u'task_logfile': u'',
  u'task_macro': ['load "Knee.any" -def N_STEP="10"',
                  'operation Main.MyStudy.InverseDynamics',
                  'classoperation Main.MyStudy.Output.MaxMuscleActivity "Dump"',
                  'run'],
  u'task_macro_hash': 2122630419,
  u'task_name': 'AnyPyTools/Tutorial_0',
  u'task_processtime': 0.5047340406127887,
  u'task_work_dir': 'E:\\Users\\MEL\\GitHub\\AnyPyTools\\Tutorial'}]

The effect is that the result of an analysis can be saved to files and later restarted.  The next example illustrates this.

## Example: Saving data to disk while running

In [19]:
import shelve
from anypytools.abcutils import AnyPyProcess
from anypytools.generate_macros import MonteCarloMacroGenerator

In [20]:
app = AnyPyProcess(return_task_info=True, disp=False)
mg = MonteCarloMacroGenerator(number_of_macros = 500)
mg.add_load('knee.any', define_kw = {'N_STEP':20})
mg.add_set_value_random_norm('Main.MyModel.Tibia.Knee.sRel', means= [0, 0.18, 0], stdvs = [0.005, 0.005, 0.005]  )
mg.add_set_value_random_norm('Main.MyModel.Femur.Knee.sRel', means= [0, -0.3, 0], stdvs = [0.005, 0.005, 0.005]   )

mg.add_run_operation('Main.MyStudy.InverseDynamics')
mg.add_dump('Main.MyStudy.Output.MaxMuscleActivity')

for macros in mg.generate_macros(batch_size=40):
    result = app.start_macro(macros)
    
    db = shelve.open('data.db', writeback=True)
    if 'result' not in db:
        db['result'] = result
    else:
        db['result'].extend(result)
    print('Saving data...')
    db.close()
    
    

Saving data...
Saving data...
Saving data...
Saving data...
Saving data...
Saving data...
Saving data...
Saving data...
Saving data...
Saving data...
Saving data...
Saving data...
Saving data...


All this stored data can be be reloaded 

In [21]:
import shelve
db = shelve.open('data.db', writeback=True)
reloaded_results = db['result']
db.close()
print('Entries in file: ' + str( len(reloaded_results)) ) 

Entries in file: 2000


In [24]:
reloaded_results[9:10]

[OrderedDict([('Main.MyStudy.Output.MaxMuscleActivity', array([ 0.01052604,  0.01518157,  0.02664641,  0.04458895,  0.06630205,
        0.08533768,  0.09494379,  0.09671828,  0.09677461,  0.09543121,
        0.09543133,  0.09677443,  0.09671828,  0.09494378,  0.08533768,
        0.06630204,  0.04458895,  0.02664643,  0.01518156,  0.01052604])), (u'task_macro_hash', -967092606), (u'task_id', 9), (u'task_work_dir', 'E:\\Users\\MEL\\GitHub\\AnyPyTools\\Tutorial'), (u'task_name', 'AnyPyTools/Tutorial_9'), (u'task_processtime', 0.810928751564461), (u'task_macro', [u'load "knee.any" -def N_STEP="20"', u'classoperation Main.MyModel.Tibia.Knee.sRel "Set Value" --value="{-0.00156888623586,0.174552561595,-0.0100680095499}"', u'classoperation Main.MyModel.Femur.Knee.sRel "Set Value" --value="{0.00446287787599,-0.309774930053,0.00840877552256}"', u'operation Main.MyStudy.InverseDynamics', u'run', u'classoperation Main.MyStudy.Output.MaxMuscleActivity "Dump"']), (u'task_logfile', u'')])]